# Feinschmecker

In [118]:
import json

from owlready2 import *
from tornado.gen import UnknownKeyError
from tornado.web import MissingArgumentError


In [119]:
NAMESPACE = "https://jaron.sprute.com/uni/actionable-knowledge-representation/feinschmecker"
# Destroy influence from last python runtime
onto = get_ontology(NAMESPACE + "/")
onto.destroy(update_relation=True, update_is_a=True)
onto = get_ontology(NAMESPACE + "/")

onto.metadata.comment.append("This project is about recipes that are used for meal preparations found in the web.")
onto.metadata.comment.append("This ontology was made by Jaron Sprute, Bhuvenesh Verma and Szymon Czajkowski.")
onto.metadata.versionInfo.append("Version: 1.1 - Existing and working ontology with sparse individuals, initial feedback included")


### Factory creations

In [120]:
def ThingFactory(name, BaseClass=Thing) -> type[Thing]:
    with onto:
        return type[Thing](name, (BaseClass,), {})


def RelationFactory(name, domain: list[ThingClass] = None, range=None) -> type[ObjectProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [Thing]
    with onto:
        return type[ObjectProperty](name, (ObjectProperty,), {
            "domain": domain,
            "range": range,
        })


def DataFactory(name, domain: list[ThingClass] = None, range=None, BaseClass=DataProperty) -> type[DataProperty]:
    if domain is None:
        domain = [Thing]
    if range is None:
        range = [str]
    with onto:
        return type[BaseClass](name, (BaseClass,), {
            "domain": domain,
            "range": range,
        })


def makeInverse(first: ObjectProperty, second: ObjectProperty) -> None:
    if first is None or second is None:
        raise TypeError("There is no inverse of no element: first:", str(first), "second:", str(second))
    with onto:
        first.inverse_property = second
        second.inverse_property = first


### Class creations

In [121]:
Recipe = ThingFactory("Recipe")
Recipe.comment = "A list of steps needed to prepare a meal, along with time of preparation, amount of calories, amounts of macroelements and information about it being vegetarian or vegan"
Ingredient = ThingFactory("Ingredient")
Ingredient.comment = "The ingredient type or name used within a recipe."
IngredientWithAmount = ThingFactory("IngredientWithAmount")
IngredientWithAmount.comment = "Technically required class in order to save the amount of a certain ingredient required for a certain recipe"
Author = ThingFactory("Author")
Author.comment = "The name (or username) of the author of the recipe written on a certain website."
Source = ThingFactory("Source")
Source.comment = "The website name including URL on which the recipe is found."
Time = ThingFactory("Time")
Time.comment = "The time required to finish the preparations of a meal according to a specific recipe. The unit is minutes."
MealType = ThingFactory("MealType")
MealType.comment = "The meal type meals can be categorized into. This includes \"Dinner\", \"Lunch\" and \"Breakfast\". Not every recipe can be categorized to belong to one of the categories."
Difficulty = ThingFactory("Difficulty")
Difficulty.comment = "A difficulty level a certain the preparation of a certain recipe is categorized as. It is differentiated between 1(easy), 2(moderate) and 3(difficult). This is a differentiated number depending on the required time and number of ingredients."

Nutrients = ThingFactory("Nutrients")
Nutrients.comment = "The nutrients one can expect to find within the meal according to a certain recipe, divided into Proteins, Fats, Carbohydrates and optionally Calories."
Calories = ThingFactory("Calories", BaseClass=Nutrients)
Calories.comment = "The calories one can expect to find within the meal according to a certain recipe. The unit is kilocalorie."
Protein = ThingFactory("Protein", BaseClass=Nutrients)
Protein.comment = "The protein one can expect to find within the meal according to a certain recipe. The unit is gram."
Fat = ThingFactory("Fat", BaseClass=Nutrients)
Fat.comment = "The fat one can expect to find within the meal according to a certain recipe. The unit is gram."
Carbohydrates = ThingFactory("Carbohydrates", BaseClass=Nutrients)
Carbohydrates.comment = "The carbohydrates one can expect to find within the meal according to a certain recipe. The unit is gram."

### Relation creations

In [122]:
# Descends according to the class creations above

# Meta classes
has_name = DataFactory("has_name", range=[str])
has_name.comment = "A meta class of all descriptions that are identical to their name."
has_amount = DataFactory("has_amount", range=[float])
has_amount.comment = "A meta class of all descriptions that are identical to their amount."

# Recipe
has_recipe_name = DataFactory("has_recipe_name", domain=[Recipe], range=[str], BaseClass=has_name)
has_recipe_name.comment = "A description of the name/title of the recipe."
has_instructions = DataFactory("has_instructions", domain=[Recipe], range=[str])
has_instructions.comment = "A description of the instructions one has to follow according to the recipe in form of a text."

has_ingredient = RelationFactory("has_ingredient", domain=[Recipe], range=[IngredientWithAmount])
has_ingredient.comment = "A link from the recipe to one of the ingredients including their amount required for it."
authored_by = RelationFactory("authored_by", domain=[Recipe], range=[Author])
authored_by.comment = "A link from the recipe to its author."
requires_time = RelationFactory("requires_time", domain=[Recipe], range=[Time])
requires_time.comment = "A link from the recipe to the time required in order to create the in the recipe specified meal."
is_meal_type = RelationFactory("is_meal_type", domain=[Recipe], range=[MealType])
is_meal_type.comment = "A link from the recipe to the meal type of which meals can be grouped into."
has_difficulty = RelationFactory("has_difficulty", domain=[Recipe], range=[Difficulty])
has_difficulty.comment = "A link from the recipes to its difficulty."

is_vegan = DataFactory("is_vegan", domain=[Recipe], range=[bool])
is_vegan.comment = "A description of whether the recipe is vegan or not."
is_vegetarian = DataFactory("is_vegetarian", domain=[Recipe], range=[bool])
is_vegetarian.comment = "A description of whether the recipe is vegetarian or not."

has_calories = RelationFactory("has_calories", domain=[Recipe], range=[Calories])
has_calories.comment = "A link from the recipe to the amount of calories a meal created according to it approximately has"
has_protein = RelationFactory("has_protein", domain=[Recipe], range=[Protein])
has_protein.comment = "A link from the recipe to the amount of protein a meal created according to it approximately has"
has_fat = RelationFactory("has_fat", domain=[Recipe], range=[Fat])
has_fat.comment = "A link from the recipe to the amount of fat a meal created according to it approximately has"
has_carbohydrates = RelationFactory("has_carbohydrates", domain=[Recipe], range=[Carbohydrates])
has_carbohydrates.comment = "A link from the recipe to the amount of carbohydrates a meal created according to it approximately has"

has_link = DataFactory("has_link", domain=[Recipe], range=[
    str])  # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property 
has_link.comment = "A description of the URL the recipe was found on. Since URLs are not supported by owlready2 in form of a text."

# Ingredient
has_ingredient_name = DataFactory("has_ingredient_name", domain=[Ingredient], range=[str], BaseClass=has_name)
has_ingredient_name.comment = "A description of the name of the ingredient."
is_ingredient_of = RelationFactory("is_ingredient_of", domain=[Ingredient], range=[IngredientWithAmount])
is_ingredient_of.comment = "A link from the ingredient to the ingredient with amount which is required for at least one recipe."

# IngredientWithAmount
has_ingredient_with_amount_name = DataFactory("has_ingredient_with_amount_name", domain=[IngredientWithAmount],
                                              range=[str], BaseClass=has_name)
has_ingredient_with_amount_name.comment = "A description of the name of the ingredient including its amount and unit."
used_for = RelationFactory("used_for", domain=[IngredientWithAmount], range=[Recipe])
used_for.comment = "A link from the ingredient with amount to the recipe which requires that specific amount of the ingredient."
type_of_ingredient = RelationFactory("type_of_ingredient", domain=[IngredientWithAmount], range=[Ingredient])
type_of_ingredient.comment = "A link from the ingredient with amount to the ingredient it has the amount of."
amount_of_ingredient = DataFactory("amount_of_ingredient", domain=[IngredientWithAmount], range=[float],
                                   BaseClass=has_amount)
amount_of_ingredient.comment = "A description of the numeric quantity the ingredient with amount has the ingredient as a floating point number."
unit_of_ingredient = DataFactory("unit_of_ingredient", domain=[IngredientWithAmount], range=[str])
unit_of_ingredient.comment = "A description of the unit the ingredient with amount has the ingredient as a text."

# Author
has_author_name = DataFactory("has_author_name", domain=[Author], range=[str], BaseClass=has_name)
has_author_name.comment = "A description of the name of the author."
authored = RelationFactory("authored", domain=[Author], range=[Recipe])
authored.comment = "A link from the author to the recipe the author authored."
is_author_of = RelationFactory("is_author_of", domain=[Author], range=[Source])
is_author_of.comment = "A link from the author in form of a user to the source/website he is part of."

# Source
has_source_name = DataFactory("has_source_name", domain=[Source], range=[str], BaseClass=has_name)
has_source_name.comment = "A description of the name of the source/website."
has_author = RelationFactory("has_author", domain=[Source], range=[Author])
has_author.comment = "A link from the source/website to the author in form of a user using this website."
is_website = DataFactory("is_website", domain=[Source], range=[
    str])  # RDF.HTML is not supported, see https://owlready2.readthedocs.io/en/latest/properties.html#data-property
is_website.comment = "A description of the URL the source/website uses. Since URLs are not supported by owlready2 in form of a text."

# Time
time_required_by = RelationFactory("time_required_by", domain=[Time], range=[Recipe])
time_required_by.comment = "A link from the time to the recipe which requires the time to complete its meal."
amount_of_time = DataFactory("amount_of_time", domain=[Time], range=[int], BaseClass=has_amount)
amount_of_time.comment = "A description of the duration of the time in minutes as a number."

# MealType
meal_type_of = RelationFactory("meal_type_of", domain=[MealType], range=[Recipe])
meal_type_of.comment = "A link from the meal type to a recipe whose meal belongs to it."
has_meal_type_name = DataFactory("has_meal_type_name", domain=[MealType], range=[str], BaseClass=has_name)
has_meal_type_name.comment = "A description of the name of the meal type."

# Difficulty
difficulty_of = RelationFactory("difficulty_of", domain=[Difficulty], range=[Recipe])
difficulty_of.comment = "A link from the difficulty to a recipe having this difficulty."
has_numeric_difficulty = DataFactory("has_numeric_difficulty", domain=[Difficulty], range=[int])
has_numeric_difficulty.comment = "A description of the difficulty in form of an int from 1 to 3 according to the difficulties description."

# Nutrients
calories_of = RelationFactory("calories_of", domain=[Calories], range=[Recipe])
calories_of.comment = "A link from the calories to the recipe whose meal approximately has the amount of calories of."
amount_of_calories = DataFactory("amount_of_calories", domain=[Calories], range=[float], BaseClass=has_amount)
amount_of_calories.comment = "A description of the amount the calories has in kilocalorie as a floating point number."
protein_of = RelationFactory("protein_of", domain=[Protein], range=[Recipe])
protein_of.comment = "A link from the protein to the recipe whose meal approximately has the amount of protein of."
amount_of_protein = DataFactory("amount_of_protein", domain=[Protein], range=[float], BaseClass=has_amount)
amount_of_protein.comment = "A description of the amount the protein has in gram as a floating point number."
fat_of = RelationFactory("fat_of", domain=[Fat], range=[Recipe])
fat_of.comment = "A link from the fat to the recipe whose meal approximately has the amount of fat of."
amount_of_fat = DataFactory("amount_of_fat", domain=[Fat], range=[float], BaseClass=has_amount)
amount_of_fat.comment = "A description of the amount the fat has in gram as a floating point number."
carbohydrates_of = RelationFactory("carbohydrates_of", domain=[Carbohydrates], range=[Recipe])
carbohydrates_of.comment = "A link from the carbohydrates to the recipe whose meal approximately has the amount of carbohydrates of."
amount_of_carbohydrates = DataFactory("amount_of_carbohydrates", domain=[Carbohydrates], range=[float],
                                      BaseClass=has_amount)
amount_of_carbohydrates.comment = "A description of the amount the carbohydrates has in gram as a floating point number."

### Interrelational properties

In [123]:
# Inverses
makeInverse(has_ingredient, used_for)
makeInverse(is_ingredient_of, type_of_ingredient)
makeInverse(authored_by, authored)
makeInverse(is_author_of, has_author)
makeInverse(requires_time, time_required_by)
makeInverse(is_meal_type, meal_type_of)
makeInverse(has_difficulty, difficulty_of)
makeInverse(has_calories, calories_of)
makeInverse(has_protein, protein_of)
makeInverse(has_fat, fat_of)
makeInverse(has_carbohydrates, carbohydrates_of)

# Limitations
with onto:
    # Recipe
    Recipe.is_a.append(has_recipe_name.exactly(1, str))
    Recipe.is_a.append(has_instructions.exactly(1, str))
    Recipe.is_a.append(has_ingredient.some(IngredientWithAmount))
    Recipe.is_a.append(authored_by.exactly(1, Author))
    Recipe.is_a.append(requires_time.exactly(1, Time))
    Recipe.is_a.append(is_vegan.max(1, bool))
    Recipe.is_a.append(is_vegetarian.max(1, bool))
    Recipe.is_a.append(is_meal_type.max(1, MealType))
    Recipe.is_a.append(has_difficulty.exactly(1, Difficulty))
    Recipe.is_a.append(has_calories.exactly(1, Calories))
    Recipe.is_a.append(has_protein.exactly(1, Protein))
    Recipe.is_a.append(has_fat.exactly(1, Fat))
    Recipe.is_a.append(has_carbohydrates.exactly(1, Carbohydrates))
    Recipe.is_a.append(has_link.exactly(1, str))

    # Ingredient
    Ingredient.is_a.append(has_ingredient_name.exactly(1, str))

    # RecipeIngredientRelation
    IngredientWithAmount.is_a.append(has_ingredient_with_amount_name.exactly(1, str))
    IngredientWithAmount.is_a.append(used_for.some(Recipe))
    IngredientWithAmount.is_a.append(type_of_ingredient.exactly(1, Ingredient))
    IngredientWithAmount.is_a.append(amount_of_ingredient.exactly(1, float))
    IngredientWithAmount.is_a.append(unit_of_ingredient.exactly(1, str))

    # Author
    Author.is_a.append(has_author_name.exactly(1, str))
    Author.is_a.append(is_author_of.exactly(1, Source))

    # Source
    Source.is_a.append(has_source_name.exactly(1, str))
    Source.is_a.append(is_website.exactly(1, str))

    # Time
    Time.is_a.append(amount_of_time.exactly(1, int))
    
    # MealType
    MealType.is_a.append(has_meal_type_name.exactly(1, str))
    
    # Difficulty
    Difficulty.is_a.append(has_numeric_difficulty.exactly(1, int))

    # Calories
    Calories.is_a.append(amount_of_calories.exactly(1, float))
    Protein.is_a.append(amount_of_protein.exactly(1, float))
    Fat.is_a.append(amount_of_fat.exactly(1, float))
    Carbohydrates.is_a.append(amount_of_carbohydrates.exactly(1, float))

# Disjointness
with onto:
    AllDisjoint([Recipe, Ingredient, IngredientWithAmount, Author, Source, Time, MealType, Difficulty, Nutrients])
    AllDisjoint([Calories, Protein, Fat, Carbohydrates])

### Individuals from JSON file

In [124]:
## Methods
def onthologifyName(name) -> str:
    return str(name).lower().replace(" ", "_").replace("%", "percent").replace("&", "and")

# Creates individual and returns individual and wether it already existed(existing -> bool = 1) with a fitting type
def createIndividual(name, BaseClass, unique=False) -> (Thing, bool):
    name = onthologifyName(name)
    if onto[name] is None:
        return BaseClass(name), False
    if type(onto[name]) != BaseClass or unique:
        raise TypeError(
            "Individual " + name + " already exists:\nExisting: " + str(type(onto[name])) + "\nRequested: " + str(
                BaseClass))
    return onto[name], True


## Static definitions
# Meal types
meal_type_names = ["Dinner", "Lunch", "Breakfast"]
meal_types = {}
for meal_type_name in meal_type_names:
    meal_type, _ = createIndividual(meal_type_name, MealType, unique=True)
    meal_type.has_meal_type_name.append(meal_type_name)
    meal_types[meal_type_name] = meal_type

# Difficulties
difficulties = []
for i in range(1,4):
    diff, _ = createIndividual("difficulty_" + str(i), Difficulty, unique=True)
    diff.has_numeric_difficulty.append(i)
    difficulties.append(diff)


## Web-scraped information
with open("recipes.json", "r") as json_recipe:
    recipes = json.load(json_recipe)

mainSource = ("BBC GoodFood", "https://bbcgoodfood.com")
source, _ = createIndividual(mainSource[0], BaseClass=Source, unique=True)
source.has_source_name.append(mainSource[0])
source.is_website.append(mainSource[1])

for json_recipe in recipes:
    recipe, _ = createIndividual(json_recipe["title"], BaseClass=Recipe, unique=True)
    recipe.has_recipe_name.append(json_recipe["title"])
    recipe.has_instructions.append(str(json_recipe["instructions"]))

    # IngredientWithAmount
    for extendedIngredient in json_recipe["ingredients"]:
        if re.search(r'\d', extendedIngredient["id"][
            0]):  # Check if first character is digit in order to avoid being identical to the ingredient
            ingredientWithAmount, existed = createIndividual(extendedIngredient["id"], BaseClass=IngredientWithAmount)
        else:
            ingredientWithAmount, existed = createIndividual("1 " + extendedIngredient["id"],
                                                             BaseClass=IngredientWithAmount)
        if existed:
            recipe.has_ingredient.append(ingredientWithAmount)
            continue
        ingredientWithAmount.has_ingredient_with_amount_name.append(extendedIngredient["id"])
        if extendedIngredient["amount"] is not None:
            ingredientWithAmount.amount_of_ingredient.append(float(extendedIngredient["amount"]))
        else:
            ingredientWithAmount.amount_of_ingredient.append(1)
        ingredientWithAmount.unit_of_ingredient.append(str(extendedIngredient["unit"]))
        ingredient, existed = createIndividual(extendedIngredient["ingredient"], BaseClass=Ingredient)
        if not existed:
            ingredient.has_ingredient_name.append(extendedIngredient["ingredient"])
        ingredientWithAmount.type_of_ingredient.append(ingredient)
        recipe.has_ingredient.append(ingredientWithAmount)

    # Author
    author, existed = createIndividual(json_recipe["author"], BaseClass=Author)
    if not existed:
        author.has_author_name.append(json_recipe["author"])
        author.is_author_of.append(source)
    recipe.authored_by.append(author)

    # Time
    time, existed = createIndividual("time_" + str(json_recipe["time"]), BaseClass=Time)
    if not existed:
        time.amount_of_time.append(json_recipe["time"])
    recipe.requires_time.append(time)

    if "meal type" in json_recipe:
        recipe.is_meal_type.append(meal_types[json_recipe["meal type"]])
    recipe.is_vegan.append(json_recipe["vegan"])
    recipe.is_vegetarian.append(json_recipe["vegetarian"])
    if len(recipe.has_ingredient) * 3 + time.amount_of_time[0] < 20:  # Easy
        recipe.has_difficulty.append(difficulties[1-1])
    elif len(recipe.has_ingredient) * 3 + time.amount_of_time[0] < 60:  # Moderate
        recipe.has_difficulty.append(difficulties[2-1])
    else:  # Difficult
        recipe.has_difficulty.append(difficulties[3-1])

    # Nutrients
    nutrients = json_recipe["nutrients"]
    calories, existed = createIndividual("calories_" + str(nutrients["kcal"]), BaseClass=Calories)
    if not existed:
        calories.amount_of_calories.append(float(nutrients["kcal"]))
    recipe.has_calories.append(calories)

    protein, existed = createIndividual("protein_" + str(nutrients["protein"]), BaseClass=Protein)
    if not existed:
        protein.amount_of_protein.append(float(nutrients["protein"]))
    recipe.has_protein.append(protein)

    fat, existed = createIndividual("fat_" + str(nutrients["fat"]), BaseClass=Fat)
    if not existed:
        fat.amount_of_fat.append(float(nutrients["fat"]))
    recipe.has_fat.append(fat)

    carbohydrates, existed = createIndividual("carbohydrates_" + str(nutrients["carbs"]), BaseClass=Carbohydrates)
    if not existed:
        carbohydrates.amount_of_carbohydrates.append(float(nutrients["carbs"]))
    recipe.has_carbohydrates.append(carbohydrates)

    recipe.has_link.append(json_recipe["source"])



### Start the reasoner

As long as the last messages the reasoner writes aren't errors it should be working just fine.

In [125]:
#with onto:
#    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

# Since the reasoner is bad, reverse its effect in the recipes instructions
#recipes = onto.search(type=Recipe)
#for recipe in recipes:
#    for instruction in recipe.has_instructions:
#        if instruction.find("\\") >= 0:
#            recipe.has_instructions.remove(instruction)
#            break

### Individual testing area

In [126]:
with onto:
    print(list(default_world.inconsistent_classes()))
    print(len(list(map(lambda a: a.name, onto.individuals()))))

#firstRecipeName = onthologifyName("Spinach, sweet potato & lentil dhal")
#print(onto[firstRecipeName])
#print(type(onto[firstRecipeName]))
#print(onto[firstRecipeName].has_ingredient)



[]
1443


## Save ontology

In [127]:
onto.save("feinschmecker.rdf")

## Unused competency question functions (replaced by filter)

In [128]:
# Tests won't work anymore since the spinach recipe was erased

# Utility functions
def getAll(objects: [Thing], attribute) -> [object]:
    result = []
    for object in objects:
        for att in getattr(object, attribute):
            result.append(att)
    return result

def getRecipe(recipe_name = None, title = None) -> Recipe:
    recipe = None
    if recipe_name is None and title is None:
        raise MissingArgumentError("You must specify a recipe or title")
    if title is not None:
        recipe_tmp = onto.search(type=Recipe, has_recipe_name=title)
        if len(recipe_tmp) != 0:
            recipe = recipe_tmp[0]
    if recipe is not None and recipe_name is not None and onto[recipe_name] is not None and type(onto[recipe_name]) is Recipe:
        recipe = onto[recipe_name]
    if recipe is None:
        raise UnknownKeyError("No recipe found. Recipe name: ", recipe_name, ", title: ", title)
    return recipe

# Selection questions
def requiredIngredients(recipe: Recipe = None, title: str = None) -> [str]:
    if recipe is not None:
        return getAll(recipe.has_ingredient, "has_ingredient_with_amount_name")
    return getAll(getRecipe(title = title).has_ingredient, "has_ingredient_with_amount_name")

def recipesWithMaxCalories(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_calories[0] <= amount else None), onto.search(type=Calories)),
                  "calories_of")


def recipesWithMinCalories(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_calories[0] >= amount else None), onto.search(type=Calories)),
                  "calories_of")


def recipesWithMaxProtein(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_protein[0] <= amount else None), onto.search(type=Protein)),
                  "protein_of")


def recipesWithMinProtein(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_protein[0] >= amount else None), onto.search(type=Protein)),
                  "protein_of")


def recipesWithMaxFat(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_fat[0] <= amount else None), onto.search(type=Fat)), "fat_of")


def recipesWithMinFat(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_fat[0] >= amount else None), onto.search(type=Fat)), "fat_of")


def recipesWithMaxCarbohydrates(amount: float = 0) -> [Recipe]:
    return getAll(
        filter(lambda x: (x if x.amount_of_carbohydrates[0] <= amount else None), onto.search(type=Carbohydrates)),
        "carbohydrates_of")


def recipesWithMinCarbohydrates(amount: float = 0) -> [Recipe]:
    return getAll(
        filter(lambda x: (x if x.amount_of_carbohydrates[0] >= amount else None), onto.search(type=Carbohydrates)),
        "carbohydrates_of")


def recipesWithMaxTime(amount: float = 0) -> [Recipe]:
    return getAll(filter(lambda x: (x if x.amount_of_time[0] <= amount else None), onto.search(type=Time)),
                  "time_required_by")


#print(requiredIngredients(title = "Spinach, sweet potato & lentil dhal"))
#print(recipesWithMaxCalories(200))
#print(recipesWithMinProtein(30))
#print(recipesWithMaxFat(20))
#print(recipesWithMinCarbohydrates(30))
#print(recipesWithMaxTime(5))

# Binary questions

def isVegan(recipe: Recipe = None, title: str = None) -> bool:
    if recipe is not None:
        return recipe.is_vegan[0]
    return getRecipe(title = title).is_vegan[0]

def isVegetarian(recipe: Recipe = None, title: str = None) -> bool:
    if recipe is not None:
        return recipe.is_vegetarian[0]
    return getRecipe(title = title).is_vegetarian[0]

def getDifficulty(recipe: Recipe = None, title: str = None) -> bool:
    if recipe is not None:
        return recipe.has_difficulty[0].has_numeric_difficulty[0]
    return getRecipe(title = title).has_difficulty[0].has_numeric_difficulty[0]

#print(getDifficulty(title = "Spinach, sweet potato & lentil dhal"))
#print(isVegan(title = "Spinach, sweet potato & lentil dhal"))
#print(isVegetarian(title = "Spinach, sweet potato & lentil dhal"))

# Counting questions

def breakfastRecipesCount() -> int:
    return len(onto.search(type=MealType, has_meal_type_name="Breakfast")[0].meal_type_of)


def lunchRecipesCount() -> int:
    return len(onto.search(type=MealType, has_meal_type_name="Lunch")[0].meal_type_of)


def dinnerRecipesCount() -> int:
    return len(onto.search(type=MealType, has_meal_type_name="Dinner")[0].meal_type_of)

#print(breakfastRecipesCount(), lunchRecipesCount(), dinnerRecipesCount())
